# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

In [0]:
# Carregar os datasets

transform=transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

dataset_valid, dataset_train = torch.utils.data.random_split(dataset_train, [5000, 45000])

Files already downloaded and verified
Files already downloaded and verified


In [0]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True, batch_size=256)
valid_loader = DataLoader(dataset=dataset_valid, shuffle=False, batch_size=256)
test_loader = DataLoader(dataset=dataset_test, shuffle=False, batch_size=256)

In [0]:
# Definir a arquitetura MLP

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #self.conv1 = torch.nn.Conv2d(1,6,5)
        #self.conv2 = torch.nn.Conv2d(6,16,10)
        
        
        #self.conv3 = torch.nn.Conv2d(16, 16, 9)
        #self.conv4 = torch.nn.Conv2d(16,16,5)
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        
        
        
        self.pool = torch.nn.MaxPool2d(2,2)
        
        
        self.fc1 = nn.Linear(32 *16 *16 , 80) # 16 * 9 * 9
        self.fc2 = nn.Linear(80, 70)
        self.fc3= nn.Linear(70, 50)
        self.fc4= nn.Linear(50, 10)  
        self.activation_function = nn.ReLU()
        
        self.conv2_drop = nn.Dropout2d(0.5)
        
        #self.conv2_bn = nn.BatchNorm2d(20)
        
    def forward(self, x):
        x = self.activation_function(self.conv1(x))
        x = self.conv2_drop(x)
        x = self.activation_function(self.conv2(x))
        x = self.conv2_drop(x)
        
        x = self.pool(x)
        
        #print(x.shape)
        
        #x = self.activation_function(self.conv3(x))
        #x = self.conv2_drop(x)
        #x = self.activation_function(self.conv4(x))
        #x = self.conv2_drop(x)
        
        #x = self.pool(x)
        
        #print(x.shape)
        
        x = x.view(-1, 32 *16 *16) # 15 * 9 * 9
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.activation_function(self.fc4(x))
        #x = self.activation_function(self.fc5(x))
        return x

In [0]:
model = CNN()
print(model)

CNN(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=70, bias=True)
  (fc3): Linear(in_features=70, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (activation_function): ReLU()
  (conv2_drop): Dropout2d(p=0.5)
)


In [0]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

#optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

loss_fn = torch.nn.CrossEntropyLoss()

#loss_fn = torch.nn.MSELoss()

In [0]:
# Realizar o treinamento aqui

epochs = 100
one_hot = torch.eye(10)
media_losses = []
media_ac = []


for epoch in range(epochs): 
  
  model.train()
  losses = []
  correct = 0
  total = 0
  for img, category in train_loader:
        
    #zera o gradiente no começo do batch
    optimizer.zero_grad()
    
    ys = model(img)
    
    #one_hot_category = one_hot[category] # apenas para quando for usar MSELoss

    loss = loss_fn(ys, category) #one_hot_category)
    
    

    loss.backward()
    
    losses.append(loss.item())

    optimizer.step()
    
    _, predicted = torch.max(ys.data, 1) # pega o segundo argumento que retorna de torch.max
    correct += (predicted == category).sum().item()
    total += category.size(0)
    
    
    #print(one_hot_category, category)
    
  
  ac = correct/total
  #media_ac.append(ac)
   #print("epoch : {}, train loss : {:.4f}, acc_train : {:.2f}%".format(epoch, np.mean(losses), ac))
  
  print("Época: ", epoch+1," ", "Loss: ", np.mean(losses), "ac: ", ac) # importante mencionar que está sendo printada a média dos losses
    
  media_losses.append(np.mean(losses)) # losses para o plot mais embaixo
  
  

Época:  1   Loss:  2.1816225973042576 ac:  0.20113333333333333
Época:  2   Loss:  1.9977141720327465 ac:  0.3080888888888889
Época:  3   Loss:  1.8954184719107368 ac:  0.34835555555555553
Época:  4   Loss:  1.8146288401701234 ac:  0.38226666666666664
Época:  5   Loss:  1.722892396829345 ac:  0.42335555555555554
Época:  6   Loss:  1.6644379557533697 ac:  0.4494444444444444
Época:  7   Loss:  1.6193863275376232 ac:  0.4629333333333333
Época:  8   Loss:  1.5801420245658269 ac:  0.47944444444444445
Época:  9   Loss:  1.5470188226212154 ac:  0.49177777777777776
Época:  10   Loss:  1.5067824883894487 ac:  0.5064222222222222
Época:  11   Loss:  1.4847117201848463 ac:  0.5116444444444445
Época:  12   Loss:  1.4618201729926197 ac:  0.5217555555555555
Época:  13   Loss:  1.4358218495141377 ac:  0.5291555555555556
Época:  14   Loss:  1.4170664420182055 ac:  0.5361333333333334
Época:  15   Loss:  1.3935589593919842 ac:  0.5454222222222223
Época:  16   Loss:  1.375180296599865 ac:  0.54775555555555

In [0]:
#graphs

import matplotlib.pyplot as plt
plt.xlabel("Epoca")
plt.ylabel("Loss")
plt.title("Gráfico Loss")
plt.plot(media_losses)
plt.show()



In [0]:
# Avaliar o modelo aqui (no conjunto de avaliação)
correct = 0
valid_losses = []
total = 0

model.eval()

with torch.no_grad():
    for images, category in valid_loader:
                y = model(images)

                #one_hot_category = one_hot[category]
                loss = loss_fn(y, category)


                valid_losses.append(loss.item())

                _, predicted = torch.max(y.data, 1)
                correct += (predicted == category).sum().item()
                total += category.size(0)
      
#ac = correct/len(valid_loader)
ac = correct/total
print("Loss_valid: ", np.mean(valid_losses), " ", "Accuracy_valid: ", ac)

correct = 0

In [0]:
# Testar o modelo aqui 


test_losses = []
total = 0

with torch.no_grad():
    for images, category in test_loader:
                yt = model(images)

                #one_hot_category = one_hot[category]
                loss = loss_fn(yt, category)


                test_losses.append(loss.item())

                _, predicted = torch.max(yt.data, 1)
                correct += (predicted == category).sum().item()
                total += category.size(0)



ac = correct/total 

#ac = correct/len(test_loader)
print("Loss_test: ", np.mean(test_losses), " ", "Accuracy_test: ", ac)

In [0]:
# Resultados obtidos:

# ---> MLP com 1 camada(s)(20) oculta(s) de 20 perceptrons + SGD + MSELoss + ReLU(sem particionar o dataset de treino): 
###### Loss_train = 0.08425  
###### Loss_valid = 0       ###### Accuracy_valid = 0
###### Loss_test  = 0.08443 ###### Accuracy_test =  0.5806


# ---> MLP com 1 camada(s)(20) oculta(s) de 20 perceptrons + SGD + MSELoss + ReLU(particionando o dataset de treino*): 
###### Loss_train = 0.0846 
###### Loss_valid = 0.0847 ###### Accuracy_valid = 0.2844
###### Loss_test  = 0.0849 ###### Accuracy_test  = 0.2868


# ---> MLP com 1 camada(s)(20) oculta(s) de 20 perceptrons + SGD + CrossEntropyLoss* + ReLU(particionando o dataset de treino):
###### Loss_train = 1.9290 
###### Loss_valid = 1.9698 ###### Accuracy_valid = 0.336
###### Loss_test  = 1.9513 ###### Accuracy_test  = 0.3353



# ---> MLP com 1 camada(s)(20) oculta(s) de 20 perceptrons + Adam + CrossEntropyLoss* + ReLU(particionando o dataset de treino):
###### Loss_train = 1.9606 
###### Loss_valid = 2.0047 ###### Accuracy_valid = 0.3042
###### Loss_test  = 2.0014 ###### Accuracy_test  = 0.3032


# ---> MLP com 2 camada(s) oculta(s)(50 e 20) de 20 perceptrons + Adam + CrossEntropyLoss + ReLU(particionando o dataset de treino):
###### Loss_train = 1.7311 
###### Loss_valid = 1.8388 ###### Accuracy_valid = 0.3738
###### Loss_test  = 1.8464 ###### Accuracy_test  = 0.3585


# ---> MLP com 2 camada(s) oculta(s)(50 e 20) de 20 perceptrons + SGD* + MSELoss* + ReLU(particionando o dataset de treino):
###### Loss_train = 0.0936 
###### Loss_valid = 0.0938 ###### Accuracy_valid = 0.18
###### Loss_test  = 0.0934 ###### Accuracy_test  = 0.1816


# ---> MLP com 4 camada(s) ocultas(100 80 50 e 20) de 20 perceptrons + SGD + MSELoss + ReLU(particionando o dataset de treino):
###### Loss_train = 1.9197 
###### Loss_valid = 1.9108 ###### Accuracy_valid = 0.3156
###### Loss_test  = 1.9309 ###### Accuracy_test  = 0.3026


#****** ---> MLP com 4 camada(s) ocultas(100 80 50 e 20) de 20 perceptrons + Adam* + CrossEntropyLoss* + ReLU(particionando o dataset de treino):
###### Loss_train = 1.5833 
###### Loss_valid = 1.9346 ###### Accuracy_valid = 0.358
###### Loss_test  = 1.9337 ###### Accuracy_test  = 0.3708




'''testh
'''


#Conclusões

Depois de testar os diversos parâmetros da MLP é possível perceber a tamanha diversidade e opções para moldá-la de acordo com o domínio ou projeto, o que é muito bom comparado ao modo que faziamos no perceptron. No primeiro trabalho foi difícil e demorado de fazer cada neurônio separado e depois juntá-los, porém no final tinhamos a melhor resposta possível(tirando o fato de ajustar apenas o neta). 

Entretanto, na MLP, é possível alterar diversos aspectos, como a arquitetura, tipo de loss, tipo de optimizador, porém é mais fácil e rápido de implementar, graças ao pytorch. Por outro lado, se perde muito tempo achando os parâmetros certos para que a rede consiga atingir o máximo de acurácia possível(no caso esperar que o treinamento acabe). 


É preciso mencionar também que a acurácia baixou drasticamente com o particionamento dos exemplos de treinamento e avaliação. O que eu já sabia que ia acontecer, entretanto seja melhor botar mais exemplos no treinamento já que eu apenas separei alguns para a avaliação.

E sobre o modelo final que escolhi como um dos melhores, foi o que possuia 4 camadas ocultas, sendo elas respectivamente de tamanhos: 100, 80, 50, 20. Eu optei em fazer uma escada como foi recomendado em aula, justamente pelo fato do aprendizado da rede neural ser como uma, onde se aprende o básico e depois irá subir até convergir para uma saída/porta. 

Eu comecei a usar o MSELoss mas depois de tentar todas as combinações cheguei a conclusão que o CrossEntropy é muito bom para a rede, pois observei que a Loss converge muito mais rápido do que o MSE. E também como vimos em aula o CrossEntropy é o mais recomendado para problemas de classificação.

O otimizador que estou usando é o Adam, que foi muito bem combinado com o CrossEntropy.(Dados das redes célula a cima)

Depois de definir qual loss e qual otimizador, testei várias arquiteturas. A partir da 3 camada oculta não dava tanto avanço nos resultados. Então optei por deixar ela pequena com 3 camadas de 80, 70 e 50, respectvamente. E os resultados foram os seguintes:


#MLP(
  (fc1): Linear(in_features=1024, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=70, bias=True)
  (fc3): Linear(in_features=70, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (activation_function): ReLU()
#)

#-----


>Média do Loss no treinamento: 1.4947


>Média da Loss na Avaliação:     1.8225


>Média da Loss no Teste:            1.8141


#-----


>Acurácia na Avaliação:               39,82%

>Acurácia no Teste:                      39,1%

